# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
%load_ext sql
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)

# pd.read_sql用のコネクタ
conn = psycopg2.connect(**pgconfig)
# pd.to_sql用のcreate engine
engine = create_engine(dsl)
# MagicコマンドでSQLを書くための設定
%sql $dsl

df_customer = pd.read_sql(sql='select * from customer', con=conn)
df_category = pd.read_sql(sql='select * from category', con=conn)
df_product = pd.read_sql(sql='select * from product', con=conn)
df_receipt = pd.read_sql(sql='select * from receipt', con=conn)
df_store = pd.read_sql(sql='select * from store', con=conn)
df_geocode = pd.read_sql(sql='select * from geocode', con=conn)

# 演習問題

---
> P-001: レシート明細のデータフレーム（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [5]:
df_receipt.head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90
5,20190605,1275696000,S13003,1112,1,CS003515000195,P050102002,1,138
6,20181205,1259971200,S14024,1102,2,CS024514000042,P080101005,1,30
7,20190922,1285113600,S14040,1102,1,CS040415000178,P070501004,1,128
8,20170504,1209859200,S13020,1112,2,ZZ000000000000,P071302010,1,770
9,20191010,1286668800,S14027,1102,1,CS027514000015,P071101003,1,680


---
> P-002: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。

In [18]:
df_receipt.sort_values(['amount'], ascending=False).head(10)[['sales_ymd', 'customer_id', 'product_cd']]

,sales_ymd,customer_id,product_cd
1202,20190903,CS011415000006,P080104055
62317,20190304,ZZ000000000000,P090302118
54095,20170921,CS028605000002,P090907210
72747,20190724,ZZ000000000000,P090907209
4632,20170721,CS015515000034,P090907208
10320,20190103,ZZ000000000000,P090907208
28304,20180621,ZZ000000000000,P090907206
97294,20190209,CS021515000089,P090907206
77874,20181005,CS008513000063,P071301009
55795,20170224,CS032414000072,P071301009


---
> P-003: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。ただし、sales_ymdはsales_dateに項目名を変更しながら抽出すること。

In [22]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd']].rename(columns={'sales_ymd':'sales_date'}).head(10)

,sales_date,customer_id,product_cd
0,20181103,CS006214000001,P070305012
1,20181118,CS008415000097,P070701017
2,20170712,CS028414000014,P060101005
3,20190205,ZZ000000000000,P050301001
4,20180821,CS025415000050,P060102007
5,20190605,CS003515000195,P050102002
6,20181205,CS024514000042,P080101005
7,20190922,CS040415000178,P070501004
8,20170504,ZZ000000000000,P071302010
9,20191010,CS027514000015,P071101003


---
> P-004: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [32]:
df_receipt.query('customer_id=="CS018205000001"')[['sales_ymd', 'customer_id', 'product_cd', 'amount']]

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-005: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [33]:
df_receipt.query('customer_id=="CS018205000001" and amount >= 1000')[['sales_ymd', 'customer_id', 'product_cd', 'amount']]

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [37]:
df_receipt.query('(1000 <= amount or quantity >= 5) and customer_id=="CS018205000001"')[['sales_ymd', 'customer_id', 'product_cd', 'quantity', 'amount']]// andは後ろにしたほうが良い

,sales_ymd,customer_id,product_cd,quantity,amount
36,20180911,CS018205000001,P071401012,1,2200
9843,20180414,CS018205000001,P060104007,6,600
21110,20170614,CS018205000001,P050206001,5,990
68117,20190226,CS018205000001,P071401020,1,2200
72254,20180911,CS018205000001,P071401005,1,1100


---
> P-007: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [ ]:
df_receipt.query('customer_id=="CS018205000001" and 1000 <= amount < 2000')[['sales_ymd', 'customer_id', 'product_cd', 'amount']]

---
> P-008: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [38]:
df_receipt.query('customer_id=="CS018205000001" and product_cd!="P071401019"')[['sales_ymd', 'customer_id', 'product_cd', 'amount']]

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。

`df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [40]:
df_store.query('prefecture_cd != "13" and floor_area <= 900')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
38,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808.0


---
> P-010: 店舗データフレーム（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件だけ表示せよ。

In [48]:
df_store.query('store_cd.str.contains("S14")', engine='python').head(10)//str.contains

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
19,S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.5789,35.29642,1838.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0


---
> P-011: 顧客データフレーム（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件だけ表示せよ。

In [49]:
df_store.query('store_cd.str.endswith("1")', engine='python').head(10)

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
5,S13051,板橋大原店,13,東京都,東京都板橋区大原町,トウキョウトイタバシクオオハラチョウ,03-0123-4029,139.6980,35.76788,1045.0
13,S13031,初台店,13,東京都,東京都渋谷区初台二丁目,トウキョウトシブヤクハツダイニチョウメ,03-0123-4020,139.6862,35.67616,986.0
15,S13001,仲六郷店,13,東京都,東京都大田区仲六郷二丁目,トウキョウトオオタクナカロクゴウニチョウメ,03-1234-4007,139.7132,35.55135,1796.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
29,S13041,八王子店,13,東京都,東京都八王子市大塚,トウキョウトハチオウジシオオツカ,042-123-4026,139.4235,35.63787,810.0
36,S14021,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0


---
> P-012: 店舗データフレーム（df_store）から横浜市の店舗だけ全項目表示せよ。

In [50]:
df_store.query('address.str.startswith("神奈川県横浜市")', engine='python')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
26,S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.5758,35.54912,1657.0
40,S14042,新山下店,14,神奈川県,神奈川県横浜市中区新山下二丁目,カナガワケンヨコハマシナカクシンヤマシタニチョウメ,045-123-4047,139.6593,35.43894,1044.0


---
> P-013: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件だけ表示せよ。

In [64]:
df_customer.query('status_cd.str.contains("^[A-F]")', engine='python')

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
6,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
15,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
21,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
...,...,...,...,...,...,...,...,...,...,...,...
21943,CS002515000242,加瀬 はるみ,1,女性,1966-06-23,52,184-0015,東京都小金井市貫井北町**********,S13002,20170102,A-20100124-7
21944,CS021615000081,小柳 遥,1,女性,1950-05-21,68,259-1133,神奈川県伊勢原市東大竹**********,S14021,20150627,D-20100710-E
21945,CS013515000188,八十田 あさみ,1,女性,1965-02-25,54,275-0021,千葉県習志野市袖ケ浦**********,S12013,20150511,D-20100708-E
21948,CS015414000107,岡野 ひろ子,1,女性,1976-11-20,42,136-0076,東京都江東区南砂**********,S13015,20150917,B-20100504-A


---
> P-014: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [65]:
df_customer.query('status_cd.str.contains("[1-9]$")', engine='python').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
9,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
16,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1
22,CS015315000033,福士 璃奈子,1,女性,1983-03-17,36,135-0043,東京都江東区塩浜**********,S13015,20141024,4-20080219-3
23,CS023513000066,神戸 そら,1,女性,1961-12-17,57,210-0005,神奈川県川崎市川崎区東田町**********,S14023,20150915,5-20100524-9
24,CS035513000134,市川 美帆,1,女性,1960-03-27,59,156-0053,東京都世田谷区桜**********,S13035,20150227,8-20100711-9
27,CS001515000263,高松 夏空,1,女性,1962-11-09,56,144-0051,東京都大田区西蒲田**********,S13001,20160812,1-20100804-1
28,CS040314000027,鶴田 きみまろ,9,不明,1986-03-26,33,226-0027,神奈川県横浜市緑区長津田**********,S14040,20150122,2-20080426-4


---
> P-015: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [67]:
df_customer.query('status_cd.str.contains("^[A-F]")', engine='python').query('status_cd.str.contains("[1-9]$")', engine='python').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
68,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
71,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
122,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
144,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
178,CS001215000097,竹中 あさみ,1,女性,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
252,CS035212000007,内村 恵梨香,1,女性,1990-12-04,28,152-0023,東京都目黒区八雲**********,S13035,20151013,B-20101018-6
259,CS002515000386,野田 コウ,1,女性,1963-05-30,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20160410,C-20100127-8
293,CS001615000372,稲垣 寿々花,1,女性,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1
297,CS032512000121,松井 知世,1,女性,1962-09-04,56,210-0011,神奈川県川崎市川崎区富士見**********,S13032,20150727,A-20100103-5


---
> P-016: 店舗データフレーム（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [71]:
df_store.query("tel_no.str.contains('[0-9]{3}-[0-9]{3}-[0-9]{4}')", engine='python')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
11,S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0


---
> P-17: 顧客データフレーム（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭10件を全項目表示せよ。

In [74]:
df_customer.sort_values('birth_day', ascending=True).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
18817,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
12328,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
15682,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
15302,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
1681,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
7511,CS001814000022,鶴田 里穂,1,女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
2378,CS016815000002,山元 美紀,1,女性,1929-02-22,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
4680,CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
16070,CS005813000015,金谷 恵梨香,1,女性,1929-04-09,89,165-0032,東京都中野区鷺宮**********,S13005,20150506,0-00000000-0
6305,CS012813000013,宇野 南朋,1,女性,1929-04-09,89,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150712,0-00000000-0


---
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭10件を全項目表示せよ。

In [75]:
df_customer.sort_values('birth_day', ascending=False).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
15639,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
7468,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
10745,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
19811,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
7039,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
3670,CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
12493,CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
15977,CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
5716,CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F
15097,CS014113000008,矢口 莉緒,1,女性,2007-03-05,12,260-0041,千葉県千葉市中央区東千葉**********,S12014,20150622,3-20091108-6


---
> P-19: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [78]:
df_receipt.sort_values('amount', ascending=False)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
1202,20190903,1283472000,S14011,1192,2,CS011415000006,P080104055,1,10925
62317,20190304,1267660800,S14046,1152,2,ZZ000000000000,P090302118,1,6800
54095,20170921,1221955200,S14028,1192,2,CS028605000002,P090907210,1,5780
72747,20190724,1279929600,S13009,1192,2,ZZ000000000000,P090907209,1,5480
4632,20170721,1216598400,S13015,1112,1,CS015515000034,P090907208,1,5480
...,...,...,...,...,...,...,...,...,...
75523,20180512,1242086400,S12007,1122,2,CS007515000037,P080102001,1,10
43059,20190511,1273536000,S14033,1112,1,CS033415000146,P080102005,1,10
36210,20170112,1200096000,S14028,112,2,CS028214000018,P080102011,1,10
9952,20180123,1232668800,S13037,1192,2,CS037415000174,P080102001,1,10


---
> P-020: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [102]:
df_tmp=pd.concat([df_receipt[['customer_id', 'amount']], df_receipt['amount'].rank(method='max')], axis=1)
df_tmp.columns=['customer_id', 'amount', 'rank']
df_tmp.sort_values('rank', ascending=False)

,customer_id,amount,rank
1202,CS011415000006,10925,104681.0
62317,ZZ000000000000,6800,104680.0
54095,CS028605000002,5780,104679.0
72747,ZZ000000000000,5480,104678.0
4632,CS015515000034,5480,104678.0
...,...,...,...
75523,CS007515000037,10,170.0
43059,CS033415000146,10,170.0
36210,CS028214000018,10,170.0
9952,CS037415000174,10,170.0


---
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [105]:
df_receipt.count()

sales_ymd         104681
sales_epoch       104681
store_cd          104681
receipt_no        104681
receipt_sub_no    104681
customer_id       104681
product_cd        104681
quantity          104681
amount            104681
dtype: int64

---
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [112]:
df_receipt['customer_id'].nunique()

8307

---
> P-023: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [124]:
df_receipt.groupby('store_cd').agg({'amount':'sum', 'quantity': 'sum'})

,amount,quantity
store_cd,,
S12007,638761,2099
S12013,787513,2425
S12014,725167,2358
S12029,794741,2555
S12030,684402,2403
S13001,811936,2347
S13002,727821,2340
S13003,764294,2197
S13004,779373,2390


---
> P-024: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [128]:
df_receipt.groupby('customer_id').agg({'sales_ymd':'min'}).head(10)

,sales_ymd
customer_id,
CS001113000004,20190308
CS001114000005,20180503
CS001115000010,20171228
CS001205000004,20170914
CS001205000006,20180207
...,...
CS051513000004,20190719
CS051515000002,20191025
CS052212000002,20191017


---
> P-025: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

In [134]:
df_receipt.groupby('customer_id').agg({'sales_ymd':['min']}).head(10)

,sales_ymd
,min
customer_id,
CS001113000004,20190308
CS001114000005,20180503
CS001115000010,20171228
CS001205000004,20170914
CS001205000006,20180207
CS001211000025,20190322
CS001212000027,20170127
CS001212000031,20180906


---
> P-026: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）と古い売上日を求め、両者が異なるデータを10件表示せよ。

In [156]:
df_tmp=df_receipt.groupby('customer_id').agg({'sales_ymd':['max', 'min']})
df_tmp.columns=['1'.join(coltpl) for coltpl in df_tmp.columns] #列名つけたい、ちなみに.だとだめ
df_tmp.query('sales_ymd1max != sales_ymd1min').head(10)

,sales_ymd1max,sales_ymd1min
customer_id,,
CS001114000005,20190731,20180503
CS001115000010,20190405,20171228
CS001205000004,20190625,20170914
CS001205000006,20190224,20180207
CS001214000009,20190902,20170306
CS001214000017,20191006,20180828
CS001214000048,20190929,20171109
CS001214000052,20190617,20180208
CS001215000005,20181021,20170206


---
> P-027: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [171]:
df_receipt.groupby('store_cd').agg({'amount':'mean'}).sort_values('amount', ascending=False).head(5) #aggでリストにしなければ列名は更新されない

,amount
store_cd,
S13052,402.867470
S13015,351.111960
S13003,350.915519
S14010,348.791262
S13001,348.470386


---
> P-028: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [172]:
df_receipt.groupby('store_cd').agg({'amount':'mean'}).sort_values('amount', ascending=False).head(5)

,amount
store_cd,
S13052,402.867470
S13015,351.111960
S13003,350.915519
S14010,348.791262
S13001,348.470386


---
> P-029: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求めよ。

In [182]:
df_receipt.groupby('store_cd').product_cd.apply(lambda x: x.mode()).reset_index() #modeはaggではできない

,store_cd,level_1,product_cd
0,S12007,0,P060303001
1,S12013,0,P060303001
2,S12014,0,P060303001
3,S12029,0,P060303001
4,S12030,0,P060303001
5,S13001,0,P060303001
6,S13002,0,P060303001
7,S13003,0,P071401001
8,S13004,0,P060303001
9,S13005,0,P040503001


---
> P-030: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本分散を計算し、降順でTOP5を表示せよ。

In [185]:
df_receipt.groupby('store_cd').agg({'amount':'std'}).sort_values('amount', ascending=False).head(5)

,store_cd,amount
0,S13052,664.727954
1,S14011,553.572256
2,S14034,545.039808
3,S13001,543.653237
4,S13015,543.532149


---
> P-031: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本標準偏差を計算し、降順でTOP5を表示せよ。

In [9]:
df_receipt.groupby('store_cd').amount.std().reset_index().sort_values('amount', ascending=False).head(5)

,store_cd,amount
28,S13052,664.727954
31,S14011,553.572256
42,S14034,545.039808
5,S13001,543.653237
12,S13015,543.532149


---
> P-032: レシート明細データフレーム（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。//パーセンタイル値は、昇順に並べたときの位置の値

In [2]:
[df_receipt['amount'].quantile(i) for i in [0.25, 0.5, 0.75]]

[102.0, 170.0, 288.0]

---
> P-033: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [4]:
df_receipt.groupby('store_cd').agg({'amount':'mean'}).query('amount >= 330')

,amount
store_cd,
S12013,330.194130
S13001,348.470386
S13003,350.915519
S13004,330.943949
S13015,351.111960
S13019,330.208616
S13020,337.879932
S13052,402.867470
S14010,348.791262


---
> P-034: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [17]:
df_receipt.groupby('customer_id').agg({'amount':'sum'}).reset_index().query('customer_id.str.contains("^(?!Z)")', engine='python')['amount'].mean()

2547.742234529256

---
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは10件だけ表示させれば良い。

In [9]:
amount_mean = df_receipt.groupby('customer_id').agg({'amount':'sum'}).reset_index().query('customer_id.str.contains("^(?!Z)")', engine='python')['amount'].mean()
df_receipt.groupby('customer_id').agg({'amount':'sum'}).reset_index().query('customer_id.str.contains("^(?!Z)")', engine='python')[df_receipt['amount'] >= amount_mean].head(10)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,customer_id,amount
89,CS001413000412,256
117,CS001414000276,278
581,CS002513000048,326
596,CS002513000319,732
1202,CS004415000012,6019
1386,CS004515000401,546
2087,CS008414000013,2523
2715,CS011415000219,2484
2821,CS011615000108,898
2901,CS012415000030,5620


---
> P-036: レシート明細データフレーム（df_receipt）と店舗データフレーム（df_store）を内部結合し、レシート明細データフレームの全項目と店舗データフレームの店舗名（store_name）を10件表示させよ。

In [21]:
pd.merge(df_receipt, df_store[['store_cd', 'store_name']], on='store_cd', how='inner')

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店
1,20181116,1258329600,S14006,112,2,ZZ000000000000,P080401001,1,48,葛が谷店
2,20170118,1200614400,S14006,1162,1,CS006815000006,P050406035,1,220,葛が谷店
3,20190524,1274659200,S14006,1192,1,CS006514000034,P060104003,1,80,葛が谷店
4,20190419,1271635200,S14006,112,2,ZZ000000000000,P060501002,1,148,葛が谷店
...,...,...,...,...,...,...,...,...,...,...
104676,20190821,1282348800,S13052,1102,1,ZZ000000000000,P071101001,1,278,森野店
104677,20190803,1280793600,S13052,1162,1,ZZ000000000000,P071005011,1,102,森野店
104678,20190714,1279065600,S13052,1132,2,ZZ000000000000,P080105001,1,135,森野店
104679,20190916,1284595200,S13052,1102,1,ZZ000000000000,P070709001,1,98,森野店


---
> P-037: 商品データフレーム（df_product）とカテゴリデータフレーム（df_category）を内部結合し、商品データフレームの全項目とカテゴリデータフレームの小区分名（category_small_name）を10件表示させよ。

In [22]:
 pd.merge(df_product, df_category[['category_small_cd', 'category_small_name']], how='inner').head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
0,P040101001,04,0401,040101,198.0,149.0,弁当類
1,P040101002,04,0401,040101,218.0,164.0,弁当類
2,P040101003,04,0401,040101,230.0,173.0,弁当類
3,P040101004,04,0401,040101,248.0,186.0,弁当類
4,P040101005,04,0401,040101,268.0,201.0,弁当類
5,P040101006,04,0401,040101,298.0,224.0,弁当類
6,P040101007,04,0401,040101,338.0,254.0,弁当類
7,P040101008,04,0401,040101,420.0,315.0,弁当類
8,P040101009,04,0401,040101,498.0,374.0,弁当類
9,P040101010,04,0401,040101,580.0,435.0,弁当類


---
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

In [38]:
pd.merge(df_customer, df_receipt, on='customer_id').query('gender_cd=="1" and customer_id.str.contains("^(?!Z)")', engine='python').groupby('customer_id').amount.sum().reset_index().head(10)

,customer_id,amount
0,CS001113000004,1298
1,CS001114000005,626
2,CS001115000010,3044
3,CS001211000025,456
4,CS001212000027,448
5,CS001212000031,296
6,CS001212000046,228
7,CS001212000070,456
8,CS001213000018,243
9,CS001213000020,343


---
> P-039: レシート明細データフレーム（df_receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが'Z'から始まるもの）は除外すること。

In [55]:
tmp_df=df_receipt.query('customer_id.str.contains("^(?!Z)")', engine='python').groupby('customer_id').agg({'amount':'sum', 'sales_ymd':'count'}).reset_index()
pd.merge(tmp_df.sort_values('amount', ascending=False).head(20), tmp_df.sort_values('sales_ymd', ascending=False).head(20), on='customer_id', how='outer')

,customer_id,amount_x,sales_ymd_x,amount_y,sales_ymd_y
0,CS017415000097,23086.0,40.0,23086.0,40.0
1,CS015415000185,20153.0,44.0,20153.0,44.0
2,CS031414000051,19202.0,38.0,19202.0,38.0
3,CS028415000007,19127.0,42.0,19127.0,42.0
4,CS001605000009,18925.0,18.0,NaN,NaN
5,CS010214000010,18585.0,44.0,18585.0,44.0
6,CS016415000141,18372.0,40.0,18372.0,40.0
7,CS006515000023,18372.0,30.0,NaN,NaN
8,CS011414000106,18338.0,32.0,NaN,NaN
9,CS038415000104,17847.0,28.0,NaN,NaN


---
> P-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（df_store）と商品（df_product）を直積した件数を計算せよ。

In [61]:
df_store_tmp = df_store.copy()
df_product_tmp = df_product.copy()

df_store_tmp['key']=0
df_product_tmp['key']=0

len(pd.merge(df_store_tmp, df_product_tmp, on='key', how='outer'))

531590

---
> P-041: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

In [102]:
tmp_df=df_receipt.groupby('sales_ymd').agg({'amount':'sum'}).reset_index().sort_values('sales_ymd')
tmp_df['sales_plus']=tmp_df.amount-tmp_df.shift(1).amount
tmp_df.head(10)

,sales_ymd,amount,sales_plus
0,20170101,33723,NaN
1,20170102,24165,-9558.0
2,20170103,27503,3338.0
3,20170104,36165,8662.0
4,20170105,37830,1665.0
5,20170106,32387,-5443.0
6,20170107,23415,-8972.0
7,20170108,24737,1322.0
8,20170109,26718,1981.0
9,20170110,20143,-6575.0


---
> P-042: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

In [103]:
df_sales_amount_by_date = df_receipt[['sales_ymd', 'amount']].groupby('sales_ymd').sum().reset_index()
for i in range(1, 4):
    if i == 1:
        df_lag = pd.concat([df_sales_amount_by_date, df_sales_amount_by_date.shift(i)],axis=1)
    else:
        df_lag = pd.concat([df_lag, df_sales_amount_by_date.shift(i)],axis=1)
df_lag.columns = ['sales_ymd', 'amount', 'lag_ymd_1', 'lag_amount_1', 'lag_ymd_2', 'lag_amount_2', 'lag_ymd_3', 'lag_amount_3']
df_lag.dropna().sort_values(['sales_ymd']).head(10)

,sales_ymd,amount,lag_ymd_1,lag_amount_1,lag_ymd_2,lag_amount_2,lag_ymd_3,lag_amount_3
3,20170104,36165,20170103.0,27503.0,20170102.0,24165.0,20170101.0,33723.0
4,20170105,37830,20170104.0,36165.0,20170103.0,27503.0,20170102.0,24165.0
5,20170106,32387,20170105.0,37830.0,20170104.0,36165.0,20170103.0,27503.0
6,20170107,23415,20170106.0,32387.0,20170105.0,37830.0,20170104.0,36165.0
7,20170108,24737,20170107.0,23415.0,20170106.0,32387.0,20170105.0,37830.0
8,20170109,26718,20170108.0,24737.0,20170107.0,23415.0,20170106.0,32387.0
9,20170110,20143,20170109.0,26718.0,20170108.0,24737.0,20170107.0,23415.0
10,20170111,24287,20170110.0,20143.0,20170109.0,26718.0,20170108.0,24737.0
11,20170112,23526,20170111.0,24287.0,20170110.0,20143.0,20170109.0,26718.0
12,20170113,28004,20170112.0,23526.0,20170111.0,24287.0,20170110.0,20143.0


---
> P-043： レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータフレーム（df_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [113]:
df_customer['ages']=df_customer['age'].round(-1)
ans1=pd.merge(df_receipt, df_customer).groupby(['ages', 'gender']).agg({'amount':'sum'})

df_tmp = pd.merge(df_receipt, df_customer, how ='inner', on="customer_id")
df_tmp['era'] = df_tmp['age'].apply(lambda x: math.floor(x / 10) * 10)
pd.pivot_table(df_tmp, index='era', columns='gender_cd', values='amount', aggfunc='sum').reset_index()

gender_cd,era,0,1,9
0,10,1591.0,149836.0,4317.0
1,20,72940.0,1363724.0,44328.0
2,30,177322.0,693047.0,50441.0
3,40,19355.0,9320791.0,483512.0
4,50,54320.0,6685192.0,342923.0
5,60,272469.0,987741.0,71418.0
6,70,13435.0,29764.0,2427.0
7,80,46360.0,262923.0,5111.0
8,90,NaN,6260.0,NaN


---
> P-044： 前設問で作成した売上サマリデータフレーム（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータフレームから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を'00'、女性を'01'、不明を'99'とする。

In [115]:
df_tmp=pd.merge(df_receipt, df_customer, how='inner', on='customer_id')
df_tmp['era'] = df_tmp['age'].apply(lambda x: math.floor(x / 10) * 10)
pd.pivot_table(df_tmp, columns='era', index='gender_cd', values='amount', aggfunc='sum').reset_index()

era,gender_cd,10,20,30,40,50,60,70,80,90
0,0,1591.0,72940.0,177322.0,19355.0,54320.0,272469.0,13435.0,46360.0,NaN
1,1,149836.0,1363724.0,693047.0,9320791.0,6685192.0,987741.0,29764.0,262923.0,6260.0
2,9,4317.0,44328.0,50441.0,483512.0,342923.0,71418.0,2427.0,5111.0,NaN


---
> P-045: 顧客データフレーム（df_customer）の生年月日（birth_day）は日付型（Date）でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [130]:
df_customer['birth_day']=pd.to_datetime(df_customer['birth_day'])
df_customer[['customer_id', 'birth_day']].head(10)

,customer_id,birth_day
0,CS021313000114,1981-04-29
1,CS037613000071,1952-04-01
2,CS031415000172,1976-10-04
3,CS028811000001,1933-03-27
4,CS001215000145,1995-03-29
5,CS020401000016,1974-09-15
6,CS015414000103,1977-08-09
7,CS029403000008,1973-08-17
8,CS015804000004,1931-05-02
9,CS033513000180,1962-07-11


---
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [154]:
pd.concat([df_customer[['customer_id']], pd.to_datetime(df_customer['application_date'])], axis=1).head(10)

,customer_id,application_date
0,CS021313000114,2015-09-05
1,CS037613000071,2015-04-14
2,CS031415000172,2015-05-29
3,CS028811000001,2016-01-15
4,CS001215000145,2017-06-05
5,CS020401000016,2015-02-25
6,CS015414000103,2015-07-22
7,CS029403000008,2015-05-15
8,CS015804000004,2015-06-07
9,CS033513000180,2015-07-28


---
> P-047: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [152]:
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']], pd.to_datetime(df_receipt['sales_ymd'].astype('str'))], axis=1).head(10)

,receipt_no,receipt_sub_no,sales_ymd
0,112,1,2018-11-03
1,1132,2,2018-11-18
2,1102,1,2017-07-12
3,1132,1,2019-02-05
4,1102,2,2018-08-21
5,1112,1,2019-06-05
6,1102,2,2018-12-05
7,1102,1,2019-09-22
8,1112,2,2017-05-04
9,1102,1,2019-10-10


---
> P-048: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [163]:
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']], pd.to_datetime(df_receipt['sales_epoch'], unit='s')], axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,2009-11-03
1,1132,2,2009-11-18
2,1102,1,2008-07-12
3,1132,1,2010-02-05
4,1102,2,2009-08-21
5,1112,1,2010-06-05
6,1102,2,2009-12-05
7,1102,1,2010-09-22
8,1112,2,2008-05-04
9,1102,1,2010-10-10


---
> P-049: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"年"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [172]:
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']], pd.to_datetime(df_receipt['sales_epoch'], unit='s').dt.year], axis=1)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,2009
1,1132,2,2009
2,1102,1,2008
3,1132,1,2010
4,1102,2,2009
...,...,...,...
104676,1132,2,2009
104677,1132,2,2010
104678,1122,1,2008
104679,1142,1,2008


---
> P-050: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"月"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"月"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [174]:
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']], pd.to_datetime(df_receipt['sales_epoch'], unit='s').dt.strftime('%m')], axis=1)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,11
1,1132,2,11
2,1102,1,07
3,1132,1,02
4,1102,2,08
...,...,...,...
104676,1132,2,02
104677,1132,2,09
104678,1122,1,03
104679,1142,1,03


---
> P-051: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"日"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"日"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [8]:
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']], pd.to_datetime(df_receipt['sales_epoch'], unit='s').dt.strftime('%d')], axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,03
1,1132,2,18
2,1102,1,12
3,1132,1,05
4,1102,2,21
5,1112,1,05
6,1102,2,05
7,1102,1,22
8,1112,2,04
9,1102,1,10


---
> P-052: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2000円以下を0、2000円超を1に2値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [31]:
df=df_receipt.query('customer_id.str.contains("^(?!Z)")', engine='python').groupby('customer_id').amount.sum().reset_index()
df['amount_idx']=df['amount'].apply(lambda x: 1 if x>2000 else 0)
pd.concat([df[['customer_id', 'amount']] ,df['amount_idx']], axis=1).head(10)

,customer_id,amount,amount_idx
0,CS001113000004,1298,0
1,CS001114000005,626,0
2,CS001115000010,3044,1
3,CS001205000004,1988,0
4,CS001205000006,3337,1
5,CS001211000025,456,0
6,CS001212000027,448,0
7,CS001212000031,296,0
8,CS001212000046,228,0
9,CS001212000070,456,0


---
> P-053: 顧客データフレーム（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に２値化せよ。さらにレシート明細データフレーム（df_receipt）と結合し、全期間において買い物実績のある顧客数を、作成した2値ごとにカウントせよ。

In [25]:
df_customer['area_cd']=df_customer['postal_cd'].apply(lambda x: 1 if 100<=int(x[0:3])<=209 else 0)
pd.merge(df_receipt['customer_id'], df_customer[['customer_id', 'area_cd']], how='left').groupby('area_cd').count()

,customer_id
area_cd,
0.0,34438
1.0,31244


---
> P-054: 顧客データデータフレーム（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに抽出せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。結果は10件表示させれば良い。

In [43]:
pd.concat([df_customer[[ 'customer_id', 'address']], df_customer['address'].str[0:3].map({'埼玉県':11, '千葉県':12, '東京都':13, '神奈川':14})], axis=1).head(10)

,customer_id,address,address
0,CS021313000114,神奈川県伊勢原市粟窪**********,14
1,CS037613000071,東京都江東区南砂**********,13
2,CS031415000172,東京都渋谷区代々木**********,13
3,CS028811000001,神奈川県横浜市泉区和泉町**********,14
4,CS001215000145,東京都大田区仲六郷**********,13
5,CS020401000016,東京都板橋区若木**********,13
6,CS015414000103,東京都江東区北砂**********,13
7,CS029403000008,千葉県浦安市海楽**********,12
8,CS015804000004,東京都江東区北砂**********,13
9,CS033513000180,神奈川県横浜市旭区善部町**********,14


---
> P-055: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額と合計ともに表示せよ。カテゴリ値は上から順に1〜4とする。結果は10件表示させれば良い。
>
> - 最小値以上第一四分位未満
> - 第一四分位以上第二四分位未満
> - 第二四分位以上第三四分位未満
> - 第三四分位以上

In [65]:
qt=[df_receipt.groupby('customer_id').amount.sum().quantile(i) for i in [0.25, 0.5, 0.75]]
def amount_category(amount):
    for i in range(3):
        if amount < qt[i]:
            return i
pd.concat([df_receipt[['customer_id', 'amount']], df_receipt.groupby('customer_id').amount.sum().reset_index()['amount'].apply(lambda x: amount_category(x)).map('{:.0f}'.format)], axis=1).head(10)


,customer_id,amount,amount
0,CS006214000001,158,1
1,CS008415000097,81,1
2,CS028414000014,170,2
3,ZZ000000000000,25,2
4,CS025415000050,90,2
5,CS003515000195,138,0
6,CS024514000042,30,0
7,CS040415000178,128,0
8,ZZ000000000000,770,0
9,CS027514000015,680,0


---
> P-056: 顧客データフレーム（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに抽出せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。先頭10件を表示させればよい。

In [74]:
pd.concat([df_customer[['customer_id', 'birth_day']] ,df_customer.age.apply(lambda x : 60 if x >= 60 else round(x, -1))], axis=1).head(10)

,customer_id,birth_day,age
0,CS021313000114,1981-04-29,40
1,CS037613000071,1952-04-01,60
2,CS031415000172,1976-10-04,40
3,CS028811000001,1933-03-27,60
4,CS001215000145,1995-03-29,20
5,CS020401000016,1974-09-15,40
6,CS015414000103,1977-08-09,40
7,CS029403000008,1973-08-17,40
8,CS015804000004,1931-05-02,60
9,CS033513000180,1962-07-11,60


---
> P-057: 前問題の抽出結果と性別（gender）を組み合わせ、新たに性別×年代の組み合わせを表すカテゴリデータを作成せよ。組み合わせを表すカテゴリの値は任意とする。先頭10件を表示させればよい。

In [79]:
tmp_df=pd.concat([df_customer[['customer_id', 'birth_day']] ,df_customer.age.apply(lambda x : 60 if x >= 60 else round(x, -1))], axis=1).reset_index()
tmp_df['era_gender'] = df_customer['gender_cd'] + tmp_df['age'].astype('str')
tmp_df.head(10)

,index,customer_id,birth_day,age,era_gender
0,0,CS021313000114,1981-04-29,40,140
1,1,CS037613000071,1952-04-01,60,960
2,2,CS031415000172,1976-10-04,40,140
3,3,CS028811000001,1933-03-27,60,160
4,4,CS001215000145,1995-03-29,20,120
5,5,CS020401000016,1974-09-15,40,040
6,6,CS015414000103,1977-08-09,40,140
7,7,CS029403000008,1973-08-17,40,040
8,8,CS015804000004,1931-05-02,60,060
9,9,CS033513000180,1962-07-11,60,160


---
> P-058: 顧客データフレーム（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに抽出せよ。結果は10件表示させれば良い。

In [82]:
pd.get_dummies(df_customer[['customer_id', 'gender_cd']], columns=['gender_cd']).head(10)

,customer_id,gender_cd_0,gender_cd_1,gender_cd_9
0,CS021313000114,0,1,0
1,CS037613000071,0,0,1
2,CS031415000172,0,1,0
3,CS028811000001,0,1,0
4,CS001215000145,0,1,0
5,CS020401000016,1,0,0
6,CS015414000103,0,1,0
7,CS029403000008,1,0,0
8,CS015804000004,1,0,0
9,CS033513000180,0,1,0


---
> P-059: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに表示せよ。標準化に使用する標準偏差は、不偏標準偏差と標本標準偏差のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [84]:
df=df_receipt.query('customer_id.str.contains("^(?!Z)")', engine='python').groupby('customer_id').amount.sum().reset_index()
df['amount_s']=preprocessing.scale(df['amount'])
df.head(10)

,customer_id,amount,amount_ss
0,CS001113000004,1298,-0.459378
1,CS001114000005,626,-0.706390
2,CS001115000010,3044,0.182413
3,CS001205000004,1988,-0.205749
4,CS001205000006,3337,0.290114
5,CS001211000025,456,-0.768879
6,CS001212000027,448,-0.771819
7,CS001212000031,296,-0.827691
8,CS001212000046,228,-0.852686
9,CS001212000070,456,-0.768879


---
> P-060: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [87]:
df=df_receipt.query('customer_id.str.contains("^(?!Z)")', engine='python'). groupby('customer_id').amount.sum().reset_index()
df['amount_s']=preprocessing.minmax_scale(df['amount'])
df.head(10)

,customer_id,amount,amount_s
0,CS001113000004,1298,0.053354
1,CS001114000005,626,0.024157
2,CS001115000010,3044,0.129214
3,CS001205000004,1988,0.083333
4,CS001205000006,3337,0.141945
5,CS001211000025,456,0.016771
6,CS001212000027,448,0.016423
7,CS001212000031,296,0.009819
8,CS001212000046,228,0.006865
9,CS001212000070,456,0.016771


---
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [89]:
df=df_receipt.query('customer_id.str.contains("^(?!Z)")', engine='python'). groupby('customer_id').amount.sum().reset_index()
df['amount_log10']=np.log10(df['amount'])
df.head(10)

,customer_id,amount,amount_log10
0,CS001113000004,1298,3.113275
1,CS001114000005,626,2.796574
2,CS001115000010,3044,3.483445
3,CS001205000004,1988,3.298416
4,CS001205000006,3337,3.523356
...,...,...,...
8301,CS051212000001,336,2.526339
8302,CS051513000004,551,2.741152
8303,CS051515000002,265,2.423246
8304,CS052212000002,192,2.283301


---
> P-062: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を自然対数化(底=e）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [90]:
df=df_receipt.query('customer_id.str.contains("^(?!Z)")', engine='python'). groupby('customer_id').amount.sum().reset_index()
df['amount_log']=np.log(df['amount'])
df.head(10)

,customer_id,amount,amount_log
0,CS001113000004,1298,7.168580
1,CS001114000005,626,6.439350
2,CS001115000010,3044,8.020928
3,CS001205000004,1988,7.594884
4,CS001205000006,3337,8.112827
5,CS001211000025,456,6.122493
6,CS001212000027,448,6.104793
7,CS001212000031,296,5.690359
8,CS001212000046,228,5.429346
9,CS001212000070,456,6.122493


---
> P-063: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益額を算出せよ。結果は10件表示させれば良い。

In [92]:
df=df_product
df['profit']=df['unit_price']-df['unit_cost']
df.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,profit
0,P040101001,04,0401,040101,198.0,149.0,49.0
1,P040101002,04,0401,040101,218.0,164.0,54.0
2,P040101003,04,0401,040101,230.0,173.0,57.0
3,P040101004,04,0401,040101,248.0,186.0,62.0
4,P040101005,04,0401,040101,268.0,201.0,67.0
5,P040101006,04,0401,040101,298.0,224.0,74.0
6,P040101007,04,0401,040101,338.0,254.0,84.0
7,P040101008,04,0401,040101,420.0,315.0,105.0
8,P040101009,04,0401,040101,498.0,374.0,124.0
9,P040101010,04,0401,040101,580.0,435.0,145.0


---
> P-064: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。
ただし、単価と原価にはNULLが存在することに注意せよ。

In [101]:
df=df_product
df['profit_rate']=(df['unit_price']-df['unit_cost'])/df['unit_price']
df.dropna()
df.groupby('product_cd').profit_rate.mean()

product_cd
P040101001    0.247475
P040101002    0.247706
P040101003    0.247826
P040101004    0.250000
P040101005    0.250000
                ...   
P091503001    0.250000
P091503002    0.250000
P091503003    0.250000
P091503004    0.249558
P091503005    0.250000
Name: profit_rate, Length: 10030, dtype: float64

---
> P-065: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [106]:
df_product['new_profit']=df_product['unit_price'].apply(lambda x: round(x, 0))*0.3
df_product['new_profit_rate']=df['new_profit']/df['unit_price']
df_product[['new_profit_rate']].head(10)

,new_profit_rate
0,0.3
1,0.3
2,0.3
3,0.3
4,0.3
5,0.3
6,0.3
7,0.3
8,0.3
9,0.3


---
> P-066: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を四捨五入すること（0.5については偶数方向の丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [107]:
df_product['new_profit']=df_product['unit_price'].apply(lambda x: round(x, 1))*0.3
df_product['new_profit_rate']=df['new_profit']/df['unit_price']
df_product[['new_profit_rate']].head(10)

,new_profit_rate
0,0.3
1,0.3
2,0.3
3,0.3
4,0.3
5,0.3
6,0.3
7,0.3
8,0.3
9,0.3


---
> P-067: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [112]:
df_product['unit_cost'].apply(lambda x: np.ceil(x/0.7))

0         213.0
1         235.0
2         248.0
3         266.0
4         288.0
          ...  
10025     300.0
10026     729.0
10027    1158.0
10028    1212.0
10029    1372.0
Name: unit_cost, Length: 10030, dtype: float64

---
> P-068: 商品データフレーム（df_product）の各商品について、消費税率10%の税込み金額を求めよ。 1円未満の端数は切り捨てとし、結果は10件表示すれば良い。ただし、単価（unit_price）にはNULLが存在することに注意せよ。

In [111]:
df_product['unit_price'].apply(lambda x: np.ceil(x*1.1))

0         218.0
1         240.0
2         254.0
3         273.0
4         295.0
          ...  
10025     308.0
10026     749.0
10027    1188.0
10028    1243.0
10029    1408.0
Name: unit_price, Length: 10030, dtype: float64

---
> P-069: レシート明細データフレーム（df_receipt）と商品データフレーム（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分"07"（瓶詰缶詰）の購入実績がある顧客のみとし、結果は10件表示させればよい。

In [23]:
tmp07_df=pd.merge(df_receipt, df_product).groupby(['customer_id', 'category_major_cd']).amount.sum().reset_index().query('category_major_cd=="07"')
tmp_df=pd.merge(df_receipt, df_product).groupby('customer_id').amount.sum().reset_index()
tmp_df=pd.merge(tmp07_df, tmp_df, on='customer_id')
tmp_df['07salerate']=tmp_df.amount_x/tmp_df.amount_y*100
tmp_df.head(10)

,customer_id,category_major_cd,amount_x,amount_y,07salerate
0,CS001113000004,07,1298,1298,100.000000
1,CS001114000005,07,486,626,77.635783
2,CS001115000010,07,2694,3044,88.501971
3,CS001205000004,07,346,1988,17.404427
4,CS001205000006,07,2004,3337,60.053941
5,CS001212000027,07,200,448,44.642857
6,CS001212000031,07,296,296,100.000000
7,CS001212000046,07,108,228,47.368421
8,CS001212000070,07,308,456,67.543860
9,CS001213000018,07,145,243,59.670782


---
> P-070: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [44]:
tmp_df=pd.merge(df_receipt[['customer_id', 'sales_ymd']], df_customer[['customer_id', 'application_date']])
tmp_df.drop_duplicates('customer_id', inplace=True)
tmp_df['sales_ymd']=pd.to_datetime(tmp_df['sales_ymd'].astype('str'))
tmp_df['application_date']=pd.to_datetime(tmp_df['application_date'])
tmp_df['gonedate']=tmp_df['sales_ymd']-tmp_df['application_date']
tmp_df.head(10)

,customer_id,sales_ymd,application_date,gonedate
0,CS006214000001,2018-11-03,2015-02-01,1371 days
22,CS008415000097,2018-11-18,2015-03-22,1337 days
30,CS028414000014,2017-07-12,2015-07-11,732 days
48,CS025415000050,2018-08-21,2016-01-31,933 days
66,CS003515000195,2019-06-05,2015-03-06,1552 days
74,CS024514000042,2018-12-05,2015-10-10,1152 days
78,CS040415000178,2019-09-22,2015-06-27,1548 days
94,CS027514000015,2019-10-10,2015-11-01,1439 days
106,CS025415000134,2019-09-18,2015-07-20,1521 days
120,CS021515000126,2017-10-10,2015-05-08,886 days


---
> P-071: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

---
> P-072: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い。（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

---
> P-073: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

---
> P-074: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、当該週の月曜日付とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値でデータを保持している点に注意）。

In [99]:
tmp_df=df_receipt.copy()
tmp_df['dayofweek']=pd.to_datetime(df_receipt['sales_ymd'].astype('str')).dt.dayofweek
tmp_df['monday']=pd.to_datetime(tmp_df['sales_ymd']).astype('str')-tmp_df['dayofweek']
#tmp_df

TypeError: unsupported operand type(s) for -: 'str' and 'int'

---
> P-075: 顧客データフレーム（df_customer）からランダムに1%のデータを抽出し、先頭から10件データを抽出せよ。

In [50]:
df_customer.sample(n=round(len(df_customer)*0.01)).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
3810,CS003515000181,笠原 さとみ,1,女性,1967-09-24,51,182-0022,東京都調布市国領町**********,S13003,20150831,0-00000000-0
11832,CS038413000086,上杉 涼子,1,女性,1971-02-18,48,279-0041,千葉県浦安市堀江**********,S13038,20150414,0-00000000-0
5102,CS011513000020,大後 りえ,1,女性,1965-01-09,54,223-0052,神奈川県横浜市港北区綱島東**********,S14011,20150408,5-20090531-7
12773,CS035612000063,脇田 一代,1,女性,1956-11-20,62,156-0055,東京都世田谷区船橋**********,S13035,20141213,0-00000000-0
19974,CS007213000020,青木 めぐみ,1,女性,1993-12-12,25,285-0845,千葉県佐倉市西志津**********,S12007,20150325,0-00000000-0
18638,CS012612000011,小菅 美智子,1,女性,1954-02-28,65,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150807,0-00000000-0
6707,CS009514000080,田崎 真悠子,1,女性,1959-06-21,59,158-0097,東京都世田谷区用賀**********,S13009,20151007,9-20100913-A
11324,CS003502000019,野中 三郎,0,男性,1966-12-21,52,182-0021,東京都調布市調布ケ丘**********,S13003,20151211,0-00000000-0
13694,CS026213000004,大山 美菜,1,女性,1998-01-01,21,251-0046,神奈川県藤沢市辻堂西海岸**********,S14026,20150914,0-00000000-0
7973,CS002411000052,水野 沙知絵,1,女性,1970-11-12,48,187-0032,東京都小平市小川町**********,S13002,20150424,3-20090906-1


---
> P-076: 顧客データフレーム（df_customer）から性別（gender_cd）の割合に基づきランダムに10%のデータを層化抽出データし、性別ごとに件数を集計せよ。

In [65]:
_, df_tmp=train_test_split(df_customer, test_size=0.2, stratify=df_customer['gender_cd'])
df_tmp.groupby('gender_cd').customer_id.count()

gender_cd
0     596
1    3584
9     215
Name: customer_id, dtype: int64

---
> P-077: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を平均から3σ以上離れたものとする。結果は10件表示させれば良い。

In [73]:
temp_df=df_receipt.query('customer_id.str.contains("^(?!Z)")', engine='python').groupby('customer_id').amount.sum().reset_index()
temp_df['amount_ss']=preprocessing.scale(temp_df['amount'])
temp_df.query('amount  >= 3*abs(amount_ss)').head(10)

,customer_id,amount,amount_ss
0,CS001113000004,1298,-0.459378
1,CS001114000005,626,-0.706390
2,CS001115000010,3044,0.182413
3,CS001205000004,1988,-0.205749
4,CS001205000006,3337,0.290114
5,CS001211000025,456,-0.768879
6,CS001212000027,448,-0.771819
7,CS001212000031,296,-0.827691
8,CS001212000046,228,-0.852686
9,CS001212000070,456,-0.768879


---
> P-078: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第一四分位と第三四分位の差であるIQRを用いて、「第一四分位数-1.5×IQR」よりも下回るもの、または「第三四分位数+1.5×IQR」を超えるものとする。結果は10件表示させれば良い。

In [81]:
tmp_df=df_receipt.query('customer_id.str.contains("^(?!Z)")', engine='python').groupby('customer_id').amount.sum().reset_index()
amount_025=tmp_df['amount'].quantile(0.25)
amount_075=tmp_df['amount'].quantile(0.75)
amount_lower=amount_025-1.5*(amount_075-amount_025)
amount_higher=amount_075+1.5*(amount_075-amount_025)
tmp_df.query('amount<@amount_lower or amount>@amount_higher').head(10)

,customer_id,amount
98,CS001414000048,8584
332,CS001605000009,18925
549,CS002415000594,9568
1180,CS004414000181,9584
1558,CS005415000137,8734
1733,CS006414000001,9156
1736,CS006414000029,9179
1752,CS006415000105,10042
1755,CS006415000147,12723
1757,CS006415000157,10648


---
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [51]:
df_product.isnull().values.sum()

14

---
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [55]:
df_product_1=df_product.dropna()
df_product_1

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,P040101001,04,0401,040101,198.0,149.0
1,P040101002,04,0401,040101,218.0,164.0
2,P040101003,04,0401,040101,230.0,173.0
3,P040101004,04,0401,040101,248.0,186.0
4,P040101005,04,0401,040101,268.0,201.0
...,...,...,...,...,...,...
10025,P091503001,09,0915,091503,280.0,210.0
10026,P091503002,09,0915,091503,680.0,510.0
10027,P091503003,09,0915,091503,1080.0,810.0
10028,P091503004,09,0915,091503,1130.0,848.0


---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [10]:
len(df_product)
df_product2=df_product.fillna({'unit_price': round(np.nanmean(df_product2['unit_price'])), 'unit_cost': round(np.nanmean(df_product2['unit_cost']))})
len(df_product2)

10030

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たなdf_product_3を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [12]:
df_product3=df_product.fillna({'unit_price': round(np.nanmedian(df_product['unit_price'])), 'unit_cost': round(np.nanmedian(df_product['unit_cost']))})
df_product3

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,P040101001,04,0401,040101,198.0,149.0
1,P040101002,04,0401,040101,218.0,164.0
2,P040101003,04,0401,040101,230.0,173.0
3,P040101004,04,0401,040101,248.0,186.0
4,P040101005,04,0401,040101,268.0,201.0
...,...,...,...,...,...,...
10025,P091503001,09,0915,091503,280.0,210.0
10026,P091503002,09,0915,091503,680.0,510.0
10027,P091503003,09,0915,091503,1080.0,810.0
10028,P091503004,09,0915,091503,1130.0,848.0


---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品の小区分（category_small_cd）ごとに算出した中央値で補完した新たなdf_product_4を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [17]:
df_by=df_product.groupby('category_small_cd').agg({'unit_price':'median', 'unit_cost':'median'}).reset_index()
df_product4=df_product.fillna({'unit_price': round(np.nanmedian(df_by['unit_price'])), 'unit_cost': round(np.nanmedian(df_by['unit_cost']))})
df_product4

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,P040101001,04,0401,040101,198.0,149.0
1,P040101002,04,0401,040101,218.0,164.0
2,P040101003,04,0401,040101,230.0,173.0
3,P040101004,04,0401,040101,248.0,186.0
4,P040101005,04,0401,040101,268.0,201.0
...,...,...,...,...,...,...
10025,P091503001,09,0915,091503,280.0,210.0
10026,P091503002,09,0915,091503,680.0,510.0
10027,P091503003,09,0915,091503,1080.0,810.0
10028,P091503004,09,0915,091503,1130.0,848.0


---
> P-084: 顧客データフレーム（df_customer）の全顧客に対し、全期間の売上金額に占める2019年売上金額の割合を計算せよ。ただし、販売実績のない場合は0として扱うこと。そして計算した割合が0超のものを抽出せよ。 結果は10件表示させれば良い。また、作成したデータにNAやNANが存在しないことを確認せよ。

In [43]:
tmp_df=pd.merge(df_customer['customer_id'], df_receipt[['customer_id', 'amount', 'sales_ymd']], how='left')
tmp_df['sales_ymd']=tmp_df['sales_ymd'].astype('str')
tmp_df['sales_ymd'].fillna(0)
ans=tmp_df.query('sales_ymd.str.startswith("2019")', engine='python').groupby('customer_id').amount.sum().reset_index()
ans['amount_rate']=ans['amount']/ans['amount'].sum()*100
ans.head(10)

,customer_id,amount,amount_rate
0,CS001113000004,1298.0,0.020118
1,CS001114000005,188.0,0.002914
2,CS001115000010,578.0,0.008958
3,CS001205000004,702.0,0.010880
4,CS001205000006,486.0,0.007533
5,CS001211000025,456.0,0.007068
6,CS001212000070,456.0,0.007068
7,CS001214000009,664.0,0.010291
8,CS001214000017,2962.0,0.045908
9,CS001214000048,1889.0,0.029278


---
> P-085: 顧客データフレーム（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いて経度緯度変換用データフレーム（df_geocode）を紐付け、新たなdf_customer_1を作成せよ。ただし、複数紐づく場合は経度（longitude）、緯度（latitude）それぞれ平均を算出すること。


In [55]:
df_geocode_mean=df_geocode.groupby('postal_cd').agg({'longitude':'mean', 'latitude':'mean'})
df_customer_1=pd.merge(df_customer, df_geocode_mean, on='postal_cd')
df_customer1

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,longitude,latitude
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,139.31779,35.41358
1,CS021303000023,堀 一徳,0,男性,1980-04-05,38,259-1113,神奈川県伊勢原市粟窪**********,S14021,20160411,0-00000000-0,139.31779,35.41358
2,CS021303000007,石倉 俊二,0,男性,1987-07-04,31,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150707,0-00000000-0,139.31779,35.41358
3,CS021313000183,草野 未來,1,女性,1986-12-21,32,259-1113,神奈川県伊勢原市粟窪**********,S14021,20170611,0-00000000-0,139.31779,35.41358
4,CS021314000098,筒井 れいな,1,女性,1985-09-21,33,259-1113,神奈川県伊勢原市粟窪**********,S14021,20160901,0-00000000-0,139.31779,35.41358
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21966,CS021612000057,星 朝陽,1,女性,1950-05-07,68,259-1134,神奈川県伊勢原市八幡台**********,S14021,20150405,4-20100829-7,139.30992,35.38544
21967,CS035401000016,白木 蒼甫,0,男性,1971-05-06,47,155-0033,東京都世田谷区代田**********,S13035,20150527,0-00000000-0,139.66281,35.65271
21968,CS003612000043,竹内 那奈,1,女性,1952-07-12,66,182-0033,東京都調布市富士見町**********,S13003,20150608,0-00000000-0,139.53539,35.66020
21969,CS007612000095,会田 陽子,9,不明,1950-12-08,68,276-0043,千葉県八千代市萱田**********,S12007,20150921,4-20100704-8,140.10959,35.73419


---
> P-086: 前設問で作成した緯度経度つき顧客データフレーム（df_customer_1）に対し、申込み店舗コード（application_store_cd）をキーに店舗データフレーム（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客の緯度・経度を用いて距離（km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示すれば良い。

$$
緯度（ラジアン）：\phi \\
経度（ラジアン）：\lambda \\
距離L = 6371 * arccos(sin \phi_1 * sin \phi_2
+ cos \phi_1 * cos \phi_2 * cos(\lambda_1 − \lambda_2))
$$

In [59]:
def calc_distance(x1, y1, x2, y2):
    distance = 6371 * math.acos(math.sin(math.radians(y1)) * math.sin(math.radians(y2)) 
                       + math.cos(math.radians(y1)) * math.cos(math.radians(y2)) 
                            * math.cos(math.radians(x1) - math.radians(x2)))
tmp_df=pd.merge(df_customer_1, df_store, left_on='application_store_cd', right_on='store_cd')

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address_x,application_store_cd,application_date,...,store_cd,store_name,prefecture_cd,prefecture,address_y,address_kana,tel_no,longitude_y,latitude_y,floor_area
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,...,S14021,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0
1,CS021303000023,堀 一徳,0,男性,1980-04-05,38,259-1113,神奈川県伊勢原市粟窪**********,S14021,20160411,...,S14021,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0
2,CS021303000007,石倉 俊二,0,男性,1987-07-04,31,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150707,...,S14021,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0
3,CS021313000183,草野 未來,1,女性,1986-12-21,32,259-1113,神奈川県伊勢原市粟窪**********,S14021,20170611,...,S14021,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0
4,CS021314000098,筒井 れいな,1,女性,1985-09-21,33,259-1113,神奈川県伊勢原市粟窪**********,S14021,20160901,...,S14021,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21966,CS052315000006,多田 優,1,女性,1979-04-15,39,194-0022,東京都町田市森野**********,S13052,20181123,...,S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087.0
21967,CS052515000004,門脇 花,1,女性,1963-12-26,55,194-0022,東京都町田市森野**********,S13052,20181109,...,S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087.0
21968,CS052714000003,河原 倫子,1,女性,1943-05-17,75,194-0022,東京都町田市森野**********,S13052,20180516,...,S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087.0
21969,CS052413000011,役所 由宇,1,女性,1977-09-16,41,194-0022,東京都町田市森野**********,S13052,20181026,...,S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087.0


---
> P-087:  顧客データフレーム（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなし、1顧客1レコードとなるように名寄せした名寄顧客データフレーム（df_customer_u）を作成せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残すものとし、売上金額合計が同一もしくは売上実績の無い顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

---
> P-088: 前設問で作成したデータを元に、顧客データフレームに統合名寄IDを付与したデータフレーム（df_customer_n）を作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定

---
> P-閑話: df_customer_1, df_customer_nは使わないので削除する。

---
> P-089: 売上実績のある顧客に対し、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

---
> P-090: レシート明細データフレーム（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月のモデル構築用データを3セット作成せよ。

---
> P-091: 顧客データフレーム（df_customer）の各顧客に対し、売上実績のある顧客数と売上実績のない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

---
> P-092: 顧客データフレーム（df_customer）では、性別に関する情報が非正規化の状態で保持されている。これを第三正規化せよ。

---
> P-093: 商品データフレーム（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータフレーム（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データフレームを作成せよ。

---
> P-094: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

---
> P-095: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはCP932

---
> P-096: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

---
> P-097: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

---
> P-098: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

---
> P-099: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

---
> P-100: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

# これで１００本終わりです。おつかれさまでした！